In [1]:
import numpy as np
from pyspark import SparkConf, SparkContext
import time
from bs4 import BeautifulSoup
import re

In [2]:
app_name = "hw3_q1"
master_URL = "spark://192.168.35.128:7077"

In [3]:
start = time.time()
conf = SparkConf().setAppName(app_name).setMaster(master_URL)
sc = SparkContext(conf = conf)

In [4]:
def deal_process(data):
    data = data.flatMap(lambda x: catch_body(x))
    return data

In [5]:
def catch_body(rdd):
    soup = BeautifulSoup(rdd[1], 'html.parser')
    data_body = soup.find_all("body")
    documents = [replace_deal(body.text) for body in data_body]
    return documents

In [6]:
def replace_deal(data):
    data = re.sub("[\.\!\/_,$%^*(+\"\']+|[+\-——！，。？、~@#￥%……&*()<>]+|Reuter|\x03", "", data)
    data = re.sub("\n", " ", data)
    data = re.sub("[ ]+", " ", data)
    return data

In [7]:
def shingling(data, news_index):
    data = data.split(" ")
    data = [word for word in data if word != ""]
    shingle = []
    for index in range(len(data)-4):
        now_shingle = ""
        for num in range(2):
            now_shingle += "{} ".format(data[index+num])
        now_shingle = now_shingle[:-1]
        shingle.append([now_shingle, news_index])
    return shingle

In [8]:
def combine(data):
    shingling_word = ""
    for word in data:
        shingling_word += word
        shingling_word += " "
    shingling_word = shingling_word[:-1]
    return shingling_word

In [9]:
def create_set(shingles, shingle_count):
    shingle_set = shingles.map(lambda x: [1 if index in x[1] else 0 for index in range(N)])
    return shingle_set

In [10]:
data = deal_process(sc.wholeTextFiles("reut2-[0-9]*.sgm"))

In [11]:
data = data.zipWithIndex()
data = data.map(lambda x: shingling(x[0], x[1]))

In [12]:
news_shingle = data.flatMap(lambda x: x).groupByKey().map(lambda x: (x[0], list(x[1])))
N = data.count()
set_shingles = create_set(news_shingle, N)
set_shingles = set_shingles.coalesce(1)

In [13]:
shingles_rdd = news_shingle.map(lambda x: x[0]).coalesce(1).union(set_shingles)
#shingles_rdd.saveAsTextFile("file:///home/bigdata/Documents/output01_{}".format(time.strftime("%Y%m%d%H%M", time.localtime())))
shingles_rdd.saveAsTextFile('output01_1')

Py4JJavaError: An error occurred while calling o87.saveAsTextFile.
: org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.SafeModeException): Cannot create directory /user/bigdata/output01_1/_temporary/0. Name node is in safe mode.
Resources are low on NN. Please add or free up more resources then turn off safe mode manually. NOTE:  If you turn off safe mode before adding resources, the NN will immediately return to safe mode. Use "hdfs dfsadmin -safemode leave" to turn safe mode off.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1335)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.mkdirs(FSNamesystem.java:3874)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.mkdirs(NameNodeRpcServer.java:984)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.mkdirs(ClientNamenodeProtocolServerSideTranslatorPB.java:634)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenode

In [ ]:
print("time is {}".format(time.time() - start))

In [ ]:
sc.stop()